##### Imports, File Setup

In [56]:
#Imports
import os
import pandas as pd
import numpy as np
from openpyxl import load_workbook
import shutil

In [57]:
# Dictionaries
curve_dict = {
    'Flow':'Q',
    'Head':'H',
    'Power':'P2',
    'Efficiency':'Eta2',
    'NPSH':'NPSH'
}

In [58]:
# This is the folder/file with the curve export csv
myDir = r"C:\Users\104092\OneDrive - Grundfos\Documents\1 - PROJECTS\SPE Integration\SPE_PumpCurve"
myFile = "PumpCurves.csv"
filePath = os.path.join(myDir, myFile)

##### Function Definitions and Class Definitions

In [59]:
# Affinity Laws function
def apply_affinity_laws(row, n2, n1):
    N1, N2 = n1, n2
    Q2 = row['Q'] * (N2 / N1)
    H2 = row['H'] * (N2 / N1)**2
    NPSH2 = row['NPSH'] * (N2 / N1)**2
    P22 = row['P2'] * (N2 / N1)**3
    # P12 = row['P1'] * (N2 / N1)**3
    rpm_pump_data = row['RPM(Pump data)']
    rpm_curve_nom = row['RPM(Curve nominal)']
    return pd.Series([Q2, H2, P22, NPSH2, rpm_pump_data, rpm_curve_nom], index=['Q','H','P2','NPSH', 'RPM(Pump data)','RPM(Curve nominal)'])

In [60]:
# Create class to hold each curve by product number/trim
class Curve:
    def __init__(self, pumpModel: str, dataframe):
        self.name = pumpModel
        self.df = dataframe
        self.pn = dataframe.iloc[0]['ProductNumber']
        self.frequency = dataframe.iloc[0]['Frequency']
        self.phase = dataframe.iloc[0]['Phase']
        self.trims = self.df['RPM(Curve nominal)'].unique().tolist()
        self.trim_curves = {} 


    def __repr__(self) -> str:
        return f"Pump Model(name={self.name}, dataframe=\n{self.df})"


    def return_xy_points(self, curveType:str):
        '''
        Returns 2 lists: 1 - flow values, 2 - Head or Power or NPSH
        
        Parameters:
        curveType (str): The list of y-values. Available options are:
            - 'Head'
            - 'Power'
            - 'NPSH'
        
        Raises:
        ValueError: If the action is not one of the available options.
        '''
        curve_types = ['Head','Power','NPSH']

        if curveType not in curve_types:
            raise ValueError(f"Invalid Curve Type. Available options are: {', '.join(curve_types)}")
        else:
            x_values = self.df['Q'].tolist()
            y_values = self.df[curve_dict[curveType]].tolist()
            return x_values, y_values
    

    def create_grouped_trim_curves(self):
        '''Group entire curve df by the trim/speed column'''
        grouped = self.df.groupby('RPM(Curve nominal)')
        for group_trim, trim_df in grouped:
            self.trim_curves[group_trim] = trim_df[['Q','H','P2','NPSH','RPM(Curve nominal)','RPM(Pump data)']]


    def create_new_trim_df(self, n2):
        """
        Takes in speed n2 and applies affinity laws to max available existing trim to calculate new curve data

        Output: Adds a dataframe to self.trim_curves dictionary

        """
        # Finds max existing trim and uses that as n1
        n1 = self.max_trim
        df1 = self.trim_curves[n1]

        # Apply the affinity laws to each row of df1 to create df2
        df2 = df1.apply(apply_affinity_laws, axis=1, args=(n2,n1))

        # Add new dataframe to dictionary trim_curves
        self.trim_curves.update({n2:df2}) 

        # Update Trims Property
        self.trims = list(self.trim_curves.keys())


    @property
    def max_trim(self):
        return max(self.trims)

##### Create objects from GPI Curve Export

In [90]:
# This separates the GPI curve export by groups according to ProductNumber
df= pd.read_csv(filePath, sep=";", index_col=False, skip_blank_lines=False) # FOR SPE
df = df.replace(',','.', regex=True)

# Drop NaN rows before grouping
df_non_nan = df.dropna(how='all')

# Forward fill productname and curve nominal columns for grouping 
df_non_nan['Product name'] = df_non_nan['Product name'].ffill()
df_non_nan['RPM(Curve nominal)'] = df_non_nan['RPM(Curve nominal)'].ffill()
df_non_nan['RPM(Pump data)'] = df_non_nan['RPM(Pump data)'].ffill()

# Group by the pump model column
grouped = df_non_nan.groupby('Product name')

# Make a model from each group 
group_objects = {}
for pumpModel, group_df in grouped:
    group_object = Curve(pumpModel=pumpModel, dataframe=group_df)
    group_object.create_grouped_trim_curves()
    group_objects[(f'{pumpModel}')] = group_object
    

C:\Users\104092\AppData\Local\Temp\ipykernel_92560\3047969589.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_non_nan['Product name'] = df_non_nan['Product name'].ffill()
C:\Users\104092\AppData\Local\Temp\ipykernel_92560\3047969589.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_non_nan['RPM(Curve nominal)'] = df_non_nan['RPM(Curve nominal)'].ffill()
C:\Users\104092\AppData\Local\Temp\ipykernel_92560\3047969589.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice 

##### Creates new speed curve data using affinity laws

In [91]:
# Iterate through list of models and and create new speed dataframes at n2
n2 = 1800

for object_name in group_objects:
    model_object = group_objects[object_name]
    model_object.create_new_trim_df(n2)

##### Writes curve data to excel file

In [92]:
# # Combine DataFrames from the dictionary into a single DataFrame
# def combine_dataframes(dict_of_dfs):
#     combined_df = pd.DataFrame()
#     for speed, df in dict_of_dfs.items():
#         # Add a column for the speed
#         df['Speed'] = speed
#         # Append to the combined DataFrame
#         combined_df = pd.concat([combined_df, df], ignore_index=True)
#     return combined_df


# # Write to excel for data validation, or plot with matplotlib
# # Create Excel Sheet Tabs, 1 for each curve
# output_file = 'SPE Added Minimum Speed Curves.xlsx'

# with pd.ExcelWriter(output_file, engine='openpyxl') as writer:
#     # Loop through each curve object
#     for object_name in group_objects:
#         curve_obj = group_objects[object_name]
#         combined_df = combine_dataframes(curve_obj.trim_curves)
            
#         # Write the combined DataFrame to an Excel sheet
#         combined_df.to_excel(writer, sheet_name=object_name, index=False)   

In [ ]:
# Leverage Luke's code to write these to xml for deployment

#### Write to PSD

In [93]:
# This points to the curve PSD template to be used
templateDir = r"C:\Users\104092\OneDrive - Grundfos\Documents\3 - RESOURCES\32 GXS"
template = "SKB Blank Curve PSD - Efficiency_Metric.xlsx"
template = os.path.join(templateDir, template)

# Create a local working copy to leave template unmodified
output_psd_file = 'SPE - new min speed curves.xlsx'
workingCopy = shutil.copyfile(template, output_psd_file)
wb = load_workbook(workingCopy)      

##### Functions

In [94]:
def curveDataMatches(curveDataDict):  
    """ Create list of PNs that share curve data """

    curveDataCopy = curveDataDict.copy()
    listOfLists = []
    
    for eachPN, eachDF in curveDataDict.items():  
        listOfPNs = []
        for key, value in curveDataCopy.items():
            if value.equals(eachDF):
                listOfPNs.append(key)
        
        if listOfPNs not in listOfLists:
            listOfLists.append(listOfPNs)
    
    # print(f'list of lists: {listOfLists}')
    # print(f'list of pns: {listOfPNs}')
    
    return listOfLists

In [95]:
def extract_variable_speeds(df) -> list:
    ''' Takes curve data in dataframe format, and returns list of all unique speeds for variable speed pumps'''
    list_of_rpms = []

    for key, value in df.iterrows():
        rpm_value = value['RPM(Curve nominal)']
        if not np.isnan(rpm_value):
            list_of_rpms.append(rpm_value)
    
    list_of_rpms = sorted(set(list_of_rpms), reverse=True)

    return list_of_rpms

##### Add 1 curve tab to workbook for every unique curve found (avoids duplicating data in SKB)


In [96]:
for object_name in group_objects:
    tabName = str(object_name)
    wb.copy_worksheet(wb['NEW']).title = tabName # Creates and renames blank PSD Tab as template for each new curve tab

##### Iterate through list of unique curve names, and populate the corresponding workbook tab


In [97]:
# Fill PSD 
for object_name in group_objects:
    curveSheet = wb[object_name]   
    # Fill Curve, power/eff, NPSH data
    baseCell = 'D7' 
    first_row_offset = 3

    # Fill Speed or trim in column A
    trims = group_objects[object_name].trims
    for index, eachSpeedTrim in enumerate(trims):
        cell_name = "{}{}".format('A', 10+index)
        curveSheet[cell_name].value = int(eachSpeedTrim)
        # print(f'For PN: {uniqueCurve[0]} -> cell: {cell_name} should have value: {int(eachSpeed)}. It currently has: {curveSheet[cell_name].value}')

        curve_data_df = group_objects[object_name].trim_curves[eachSpeedTrim].reset_index()
        for key, value in curve_data_df.iterrows():
            # print(f'first_row_offset {first_row_offset}, key: {key + 21*index}')
            curveSheet[baseCell].offset(first_row_offset + key, 0 + 21*index).value = value['Q']
            curveSheet[baseCell].offset(first_row_offset + key, 1 + 21*index).value = value['H']
            curveSheet[baseCell].offset(first_row_offset + key, 7 + 21*index).value = value['Q']
            # curveSheet[baseCell].offset(first_row_offset + key, 8 + 21*index).value = value['Eta1']
            curveSheet[baseCell].offset(first_row_offset + key, 8 + 21*index).value = value['P2']
            curveSheet[baseCell].offset(first_row_offset + key,14 + 21*index).value = value['Q']
            curveSheet[baseCell].offset(first_row_offset + key,15 + 21*index).value = value['NPSH']

first_row_offset 3, key: 0
first_row_offset 3, key: 1
first_row_offset 3, key: 2
first_row_offset 3, key: 3
first_row_offset 3, key: 4
first_row_offset 3, key: 5
first_row_offset 3, key: 6
first_row_offset 3, key: 7
first_row_offset 3, key: 8
first_row_offset 3, key: 9
first_row_offset 3, key: 10
first_row_offset 3, key: 11
first_row_offset 3, key: 12
first_row_offset 3, key: 13
first_row_offset 3, key: 14
first_row_offset 3, key: 15
first_row_offset 3, key: 16
first_row_offset 3, key: 17
first_row_offset 3, key: 18
first_row_offset 3, key: 21
first_row_offset 3, key: 22
first_row_offset 3, key: 23
first_row_offset 3, key: 24
first_row_offset 3, key: 25
first_row_offset 3, key: 26
first_row_offset 3, key: 27
first_row_offset 3, key: 28
first_row_offset 3, key: 29
first_row_offset 3, key: 30
first_row_offset 3, key: 31
first_row_offset 3, key: 32
first_row_offset 3, key: 33
first_row_offset 3, key: 34
first_row_offset 3, key: 35
first_row_offset 3, key: 36
first_row_offset 3, key: 37
fi

In [78]:
# Save changes to excel sheet
wb.save(workingCopy)

##### Fill Curve Header Tab

In [98]:
for object_name in group_objects:
    df = group_objects[object_name].df
    max_speed = max(group_objects[object_name].trims)
    min_speed = min(group_objects[object_name].trims)
    
    newPumpFamilyName = "SPE"
    sheet = wb["Curve Header Data"]
    sheet['B7'] = newPumpFamilyName
    baseCell = 'A1' 
    first_row_offset = 10

    for index, speedData in df.iterrows():
        sheet[baseCell].offset((first_row_offset + index), 0).value = speedData['Product name']
        sheet[baseCell].offset((first_row_offset + index), 2).value = speedData['RPM(Pump data)']
        sheet[baseCell].offset((first_row_offset + index), 6).value = df.iloc[0]['RPM(Curve nominal)']
        sheet[baseCell].offset((first_row_offset + index), 7).value = min_speed
        sheet[baseCell].offset((first_row_offset + index), 8).value = max_speed

wb.save(workingCopy)

In [101]:
wb.close()

##### Create list of PNs that share curve data, and inserts tab to illustrate

In [80]:
# curveFamilySheet = wb.create_sheet("Shared Curves", 2)

# for index, family in enumerate(uniqueCurvePartnumbers):
#     row = index + 1  # Excel doesn't like 0-indexes

#     # Fill 1st column in spreadsheet
#     curveFamilySheet.cell(row=row, column=1).value = "Curve " + str(row)

#     # Fill 2nd (or more) columns with partnumbers that share curves
#     if len(family) == 1:
#         curveFamilySheet.cell(row=row, column=2).value = family[0]
#         # print('length of family is 1')
#     else:
#         curveFamilySheet.cell(row=row, column=2).value = family[0]
#         curveFamilySheet.cell(row=row, column=3).value = family[1]
#         # print(f'family[0]: {family[0]}, family[1]: {family[1]}')